In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)


import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

all_categories.append(0)
all_categories.append(1)

n_categories = 2

In [2]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    
    return all_categories[category_i], category_i

In [3]:
import pandas as pd


dataCSV = pd.read_csv('data/train.csv', nrows=10000)


In [4]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [5]:
input = lineToTensor(dataCSV.iloc[0, 1])
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)



tensor([[-0.7019, -0.6845]], grad_fn=<LogSoftmaxBackward>)


In [6]:
criterion = nn.NLLLoss()

In [7]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item()

In [ ]:
import time
import math

n_iters = len(dataCSV)
print_every = 100
plot_every = 1000



# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for epoca in range(0, 10):
    for iter in range(1, n_iters + 1):
        line = dataCSV.iloc[iter, 1]    
        category = dataCSV.iloc[iter, 2]

        line_tensor = lineToTensor(dataCSV.iloc[iter, 1])
        category_tensor = torch.tensor([category], dtype=torch.long)    


        output, loss = train(category_tensor, line_tensor)
        current_loss += loss

        # Print iter number, loss, name and guess
        #if iter % print_every == 0:
        if iter % print_every == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('Epoca %d - %d %d%% (%s) %.4f (%s...) / %s %s' % (epoca, iter, iter / n_iters * 100, timeSince(start), loss, line[:20], guess, correct))

        # Add current loss avg to list of losses
        if iter % plot_every == 0:
            all_losses.append(current_loss / plot_every)
            current_loss = 0

0 - 100 1% (0m 6s) 0.3413 (What do physicists, ...) / 0 ✓
0 - 200 2% (0m 13s) 0.2122 (Which location is th...) / 0 ✓
0 - 300 3% (0m 18s) 0.1530 (What is the largest ...) / 0 ✓
0 - 400 4% (0m 20s) 2.3146 (Why did the Pope say...) / 0 ✗ (1)
0 - 500 5% (0m 23s) 0.0687 (What are the reasons...) / 0 ✓
0 - 600 6% (0m 26s) 0.0544 (Do you know about Mu...) / 0 ✓
0 - 700 7% (0m 28s) 0.0344 (Is there anything ca...) / 0 ✓
0 - 800 8% (0m 30s) 0.0451 (Why are drain flies ...) / 0 ✓
0 - 900 9% (0m 32s) 0.0461 (My skin turns dark w...) / 0 ✓
0 - 1000 10% (0m 34s) 0.0467 (What is that one lif...) / 0 ✓
0 - 1100 11% (0m 37s) 0.0628 (Why is Niccolò Machi...) / 0 ✓
0 - 1200 12% (0m 39s) 0.0743 (What is the scope fo...) / 0 ✓
0 - 1300 13% (0m 41s) 0.1216 (Amanda rodgers? Who ...) / 0 ✓
0 - 1400 14% (0m 43s) 0.0821 (What is your best po...) / 0 ✓
0 - 1500 15% (0m 50s) 0.0727 (Why is there a week ...) / 0 ✓
0 - 1600 16% (0m 58s) 0.0649 (How long does it tak...) / 0 ✓
0 - 1700 17% (1m 6s) 0.0761 (How do I d

In [14]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

plt.figure()
plt.plot(all_losses)